# Objetivo

Realizar processo de coleta de dados estruturados da web de maneira automatizada (web scraping) de site de imóveis residenciais de Curitiba (Brasil).

# Procedimentos
- Importar bibliotecas
- Estabelecer conexão com o site 'portalimoveiscuritiba.com.br'
- Coletas de dados
- Criação de ficheiro

# Bibliotecas

In [1]:
# importando bibliotecas
from urllib.request import urlopen, urlretrieve, Request
from urllib.error import URLError, HTTPError
import urllib.request as urllib_request
from bs4 import BeautifulSoup
import pandas as pd

# Conexão com o site

In [13]:
# atribuindo site para variável
url = "https://portalimoveiscuritiba.com.br/imoveis?nidtbxfin=1&nidtbxtpi=1&nidtagbac=&nidtbxloc="

# teste de erros
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/76.0.3809.100 Safari/537.36'}

try:
    req = Request(url, headers = headers)
    response = urlopen(req, timeout=20)
    print(response.read())

except HTTPError as e:
    print(e.status, e.reason)

except URLError as e:
    print(e.reason)

b'<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN" "https://www.w3.org/TR/html4/loose.dtd">\n<html>\n    <head>\n    <meta http-equiv="Content-Type" content="text/html; charset=UTF-8">\n    <meta name="viewport" content="width=device-width, initial-scale=1">\n        <meta property="og:type" content="website" />\n        <meta property="og:title" content="Portal Im\xc3\xb3veis Curitiba" />\n        <meta property="og:description" content="Portal im\xc3\xb3veis curitiba. Imobili\xc3\xa1rias em Curitiba e Regi\xc3\xa3o. Apartamentos e casas para alugar em curitiba. Procurando im\xc3\xb3veis em curitiba? Veja ofertas!" />\n        <meta property="og:url" content="https://portalimoveiscuritiba.com.br/imoveis?nidtbxfin=1&nidtbxtpi=1&nidtagbac=&nidtbxloc=" />\n        <meta property="og:image" content="https://portalimoveiscuritiba.com.br/images/screenshot-portal-2.png" />\n    <meta property="og:image:width" content="291" />\n    <meta property="og:image:height" content="167" 

A conexão foi estabelecida sem erros.

# Scraping

In [44]:
# obtendo dados da HTML
response = urlopen(url, timeout=20) 
html = response.read().decode('utf-8') 

# tratamento de dados da html: eliminar espaços entre as TAGs
html = html.join(input.split()).replace('> <', '><')

# criação do objeto BeautifulSoup
soup = BeautifulSoup(html, 'html.parser')
pages = int(soup.find('div', class_='pagerfanta').get_text().split()[-1].split('...')[-1])

# declarando variável cards que armazenará conteúdo do dataset
cards = []

# iterando todas as páginas do site
for i in range(pages):
    
    # obtendo o HTML
    response = urlopen('https://portalimoveiscuritiba.com.br/imoveis?nidtbxfin=1&nidtbxtpi=1&nidtagbac=&nidtbxloc=&page=' + str(i + 1))
    html = response.read().decode('utf-8')
    soup = BeautifulSoup(html, 'html.parser')
    
    # obtendo as TAGs de interesse: todos os imóveis anunciados
    ads = soup.find('div', class_="tl-filterOuter").findAll('div', class_="row")
    
    for ad in ads:
        card = {}

        # Valor
        card['price'] = ad.find('h3', {'class': 'price'}).getText()

        # Localização
        address = ad.find('span', {'class': 'address'}).getText().split(' - ')
        card['city'] = address[0].lower()
        card['neighborhood'] = address[-1].lower()
        
        # Quartos e Garagens
        info = ad.find('ul', {'class': 'tl-meta-listed'}).get_text().split()
        card['bedrooms'] = info[1]
        card['garage'] = info[-1]
        
        # Adicionando resultado à lista cards
        cards.append(card)    

# Ficheiro CSV

In [ ]:
# Criando um DataFrame com os resultados
dataset = pd.DataFrame(cards)
dataset['bedrooms'][dataset['bedrooms'] == 'Garagens'] = None
dataset['garage'][dataset['garage'] == 'Garagens'] = None
dataset.to_csv('./imoveis.csv', sep=';', index = False, encoding = 'utf-8-sig')

In [45]:
# quantidade de cards obtidos 
len(cards)

2823

In [46]:
# quantidade de linhas e colunas do dataset
dataset.shape

(2823, 5)

In [47]:
# visualização do dataset
dataset

,price,city,neighborhood,bedrooms,garage
0,"R$ 189.000,00",araucária,iguaçu,2,1
1,"R$ 310.000,00",araucária,centro,2,1
2,"R$ 194.000,00",curitiba,santa cândida,2,1
3,"R$ 3.850.000,00",curitiba,ecoville,4,4
4,"R$ 2.712.000,00",curitiba,ecoville,3,4
...,...,...,...,...,...
2818,"R$ 399.000,00",curitiba,água verde,2,1
2819,"R$ 220.000,00",curitiba,portão,None,None
2820,"R$ 2.732.353,00",curitiba,batel,4,4
2821,"R$ 6.680.000,00",curitiba,ecoville,4,6


In [48]:
# informações sobre o dataset
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2823 entries, 0 to 2822
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   price         2823 non-null   object
 1   city          2823 non-null   object
 2   neighborhood  2823 non-null   object
 3   bedrooms      2787 non-null   object
 4   garage        2696 non-null   object
dtypes: object(5)
memory usage: 110.4+ KB


In [49]:
# estatísticas descritivas do dataset
dataset.describe()

,price,city,neighborhood,bedrooms,garage
count,2823,2823,2823,2787,2696
unique,1433,12,141,7,9
top,"R$ 220.000,00",curitiba,água verde,3,1
freq,26,2412,332,1244,1321
